<a href="https://colab.research.google.com/github/Hooleymass/Learning/blob/master/python_for_program_3a_forwardalgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Python Framework for Programming 1A through 3A

In [ ]:
# Paraemter values for the 1D keyboad are stored in global variables; you can change them
pr_hit = 0.6
pr_miss = 0.4
deg_kb = 2

pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2

# 1. Key functions implemented so far

## getPrTableForPossibleInitialStates(lengthOfWord)

In [ ]:
# C++:  getPrTableForPossibleInitialStates(prTable, lengthOfWord):
# ==>
# Python:void getPrTableForPossibleInitialStates(lengthOfWord):
#           return the information in the prTable directly
def getPrTableForPossibleInitialStates(lengthOfWord):
    missDistance = range( 1, lengthOfWord+1 )
    exponentialDegrade = [ (1/deg_sp)**i    for i in missDistance]
    scalingConstant = 1 / sum(exponentialDegrade)
    return [scalingConstant*degrade for degrade in exponentialDegrade]


# Test the function to get the probabilities of the possible first states
#      for a word of 3 characters (such as "his" in our handout)
pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2
getPrTableForPossibleInitialStates(3)

[0.5714285714285714, 0.2857142857142857, 0.14285714285714285]

## getPrTableForPossibleInitialStatesGivenTheWord(Word)

In [ ]:
# A variant that accomplishes the same thing given a word (as a string)
def getPrTableForPossibleInitialStatesGivenTheWord(Word):
    missDistance = range( 1, len(Word)+1 )
    exponentialDegrade = [ (1/deg_sp)**i    for i in missDistance]
    scalingConstant = 1 / sum(exponentialDegrade)
    return [scalingConstant*degrade for degrade in exponentialDegrade]


# Test the function to get the probabilities of the possible first states
#      for the word "his"
pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2
getPrTableForPossibleInitialStatesGivenTheWord("his")

[0.5714285714285714, 0.2857142857142857, 0.14285714285714285]

## getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState)

In [ ]:
# C++
# void getPrTableForPossibleNextStates(double transitionPrTable[],
#                                      int sizeOfTable, int currentState)
# ==>
# Python
# getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState)
# return the transitionPrTable

def getPrTableForPossibleNextStates(lengthOfWord_Plus1, currentState):
    statesAsIndices = range( lengthOfWord_Plus1 )
    distances = [state - currentState for state in statesAsIndices ]
    exponentialDegrade = [ (1/deg_sp)**i if i>0 else 0 for i in distances]
    scalingConstant = pr_moveOn/sum(exponentialDegrade)
    probabilitiesOfPossibleFirstStates = (
        [ scalingConstant*degrade for degrade in exponentialDegrade] )
    probabilitiesOfPossibleFirstStates[currentState] = pr_repeat
    return probabilitiesOfPossibleFirstStates


#Test the implementation
getPrTableForPossibleNextStates(len("his")+1, 0)

[0.2, 0.4571428571428572, 0.2285714285714286, 0.1142857142857143]

## getPrTableForPossibleNextStatesGivenWord(word, currentState)

In [ ]:
# A convenient variant
# Python
# getPrTableForPossibleNextStates(word, int currentState)
# return the transitionPrTable

def getPrTableForPossibleNextStatesGivenWord(word, currentState):
    lengthOfWord_Plus1 =  len(word) +1
    statesAsIndices = range( lengthOfWord_Plus1 )
    distances = [state - currentState for state in statesAsIndices ]
    exponentialDegrade = [ (1/deg_sp)**i if i>0 else 0 for i in distances]
    scalingConstant = pr_moveOn/sum(exponentialDegrade)
    probabilitiesOfPossibleFirstStates = (
        [ scalingConstant*degrade for degrade in exponentialDegrade] )
    probabilitiesOfPossibleFirstStates[currentState] = pr_repeat
    return probabilitiesOfPossibleFirstStates

#Test the implementation
currentState = 0
getPrTableForPossibleNextStatesGivenWord("his", currentState)

[0.2, 0.4571428571428572, 0.2285714285714286, 0.1142857142857143]

## prCharGiveCharState(x, y)

In [ ]:
#Probability of touching x when trying to type y
def prCharGiveCharState(x, y):
    if x==y:
        return pr_hit

    diffASCII = range(1,26)
    missdist = [min(n, 26-n) for n in diffASCII ]
    exponentialDegrade = [(1/deg_kb)**i for i in missdist]
    constant_x= pr_miss/sum(exponentialDegrade)

    distASCII_x_y = abs( ord(x) - ord(y) )
    distKB_x_y = min(distASCII_x_y, 26-distASCII_x_y )
    return constant_x* (1/deg_kb)** distKB_x_y

In [ ]:
pr_hit

0.6

In [ ]:
deg_kb

2

In [ ]:
prCharGiveCharState('b', 'a')

0.1000183139002503

In [ ]:
prCharGiveCharState('c', 'a')

0.05000915695012515

## take1SampleFrom1PrSpace

In [ ]:
# C++: int take1SampleFrom1PrSpace(double prTable[], int sizeOfTable)
# ==>
# Python: take1SampleFrom1PrSpace(prTable)
#  the size of the table can be implicitly determined
def take1SampleFrom1PrSpace(prTable):
    probabilityThresholds = np.add.accumulate(prTable)
    sample = np.random.random()
    choice = (sample > probabilityThresholds).sum()
    # print("Sample=", sample, ",\t choice=", choice)
    return choice


import numpy as np
prTable = np.array([0.25, 0.5, 0.25])
[take1SampleFrom1PrSpace(prTable) for i in range(20)]

[0, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 0, 0, 2]

In [ ]:
#Use bin count to check empirical frequencies observed
np.bincount( [take1SampleFrom1PrSpace(prTable) for i in range(1000000)] )/1000000

array([0.250308, 0.499671, 0.250021])

## getKeyboardProbabilityTable
### note: this is a simple variant of prCharGiveCharState(x, y)

In [ ]:
#C++: void getKeyboardProbabilityTable(char charToType, double prTable[])
#==>
#Python: getKeyboardProbabilityTable(charToType)
#        to return the probabilities of getting a, b, ..., y, z
#        as a numpy array
#Note: This is simply a simple variant of prCharGiveCharState(x, y):

def getKeyboardProbabilityTable(charToType):
    #First determine the scaling constant for the exponential degrading
    diffASCII = range(1,26)
    missdist = [min(n, 26-n) for n in diffASCII ]
    exponentialDegrade = [(1/deg_kb)**i for i in missdist]
    scalingConstant = pr_miss/sum(exponentialDegrade)

    #Set up an empty probability table
    prTable = np.empty(26)
    y = charToType

    # for each x in a to z,
    # set up a loop to determine the probability of touching x
    #     when trying to type y (i.e.charToType)
    # store the results in the probability table accordingly
    for i, x in enumerate("abcdefghijklmnopqrstuvwxyz"):
        if x==y:
            prTable[i] = pr_hit
        else:
            distASCII_x_y = abs( ord(x) - ord(y) )
            distKB_x_y = min(distASCII_x_y, 26-distASCII_x_y )
            prTable[i] = scalingConstant * (1/deg_kb)** distKB_x_y

    return prTable


#Test the implementation
pr_hit = 0.6
pr_miss = 0.4
deg_kb = 3

getKeyboardProbabilityTable('a')

array([6.00000000e-01, 1.33333501e-01, 4.44445002e-02, 1.48148334e-02,
       4.93827780e-03, 1.64609260e-03, 5.48697533e-04, 1.82899178e-04,
       6.09663926e-05, 2.03221309e-05, 6.77404362e-06, 2.25801454e-06,
       7.52671513e-07, 2.50890504e-07, 7.52671513e-07, 2.25801454e-06,
       6.77404362e-06, 2.03221309e-05, 6.09663926e-05, 1.82899178e-04,
       5.48697533e-04, 1.64609260e-03, 4.93827780e-03, 1.48148334e-02,
       4.44445002e-02, 1.33333501e-01])

In [ ]:
pr_hit

0.6

In [ ]:
#More test on the implementation
getKeyboardProbabilityTable('b')

array([1.33333501e-01, 6.00000000e-01, 1.33333501e-01, 4.44445002e-02,
       1.48148334e-02, 4.93827780e-03, 1.64609260e-03, 5.48697533e-04,
       1.82899178e-04, 6.09663926e-05, 2.03221309e-05, 6.77404362e-06,
       2.25801454e-06, 7.52671513e-07, 2.50890504e-07, 7.52671513e-07,
       2.25801454e-06, 6.77404362e-06, 2.03221309e-05, 6.09663926e-05,
       1.82899178e-04, 5.48697533e-04, 1.64609260e-03, 4.93827780e-03,
       1.48148334e-02, 4.44445002e-02])

In [ ]:
#More test on the implementation
getKeyboardProbabilityTable('b').sum()

1.0

## typeOneChar

In [ ]:
# C++: char typeOneChar(char charToType)
# Python: typeOneChar(charToType)
#   use  take1SampleFrom1PrSpace and
#        getKeyboardProbabilityTable to
#   simulate typing charToType and return the resulting character pressed

def typeOneChar(charToType):
    keys = "abcdefghijklmnopqrstuvwxyz"
    prTable = getKeyboardProbabilityTable(charToType)
    indexOfKeyPressed = take1SampleFrom1PrSpace( prTable )
    return keys[ indexOfKeyPressed ]

#Test the implementatiton
pr_hit = 0.2
pr_miss = 0.8
deg_kb = 1.2

#Type 'a' for 10 times and see the results
[typeOneChar('a') for i in range(10) ]

['a', 'a', 'c', 'i', 'y', 'c', 'b', 'y', 'g', 'a']

In [ ]:
#More test the implementatiton
pr_hit = 0.2
pr_miss = 0.8
deg_kb = 2

#Type 'a' for 10 times under a different setting and see the results
[typeOneChar('a') for i in range(10) ]

['b', 'x', 'a', 'z', 'x', 'z', 'b', 'b', 'a', 'x']

##  typeOneWord

In [ ]:
# C++: void typeOneWord( char word[], char output[],
#                        bool traceON = false, int maxOutput=100)
# Python: typeOneWord( word, trace=False )
#      simulate the typing of the given word (a string) and
#      return the resulted string

def typeOneWord(word, trace=False):
    #Special States
    I_stateIndex = -1
    F_stateIndex = len(word)

    # Step 0: Simulation of leaving the starting state I to enter some (first) state
    #        to enter a regular state as the current state: throw a dice
    charOutputsObservedSofar = "_"
    stateTrajectorySofar = "I"
    prTable = getPrTableForPossibleInitialStatesGivenTheWord(word)

    currentState_index = take1SampleFrom1PrSpace(prTable)
    currentState_char = word[ currentState_index  ]

    if trace:
        print( "First state reached after leaving I: (index, char)=",
               (currentState_index, currentState_char)
             )

    while( currentState_index != F_stateIndex): # not the Final state F yet.
        # Step 1: Simulation of typing a character given the current state: throw a dice
        charTyped = typeOneChar(currentState_char)
        charOutputsObservedSofar += charTyped
        stateTrajectorySofar += currentState_char

        if trace:
            print( "Current state: (index, char)=", (currentState_index, currentState_char) )
            print( charTyped, " is pressed when trying to type ", currentState_char)
            print( "char outputs so far:\t", charOutputsObservedSofar )
            print( "state trajectory so far:", stateTrajectorySofar )
            print()

        # Step 2: Simulation of leaving the current state to enter one of the possble next states
        #       : throw a dice
        prTable = getPrTableForPossibleNextStatesGivenWord(word, currentState_index)
        nextState_index = take1SampleFrom1PrSpace(prTable)
        nextState_char = word[ nextState_index  ] if (nextState_index<F_stateIndex) else "F"
        if trace:
            print( "next state to enter: ", (nextState_index,
                                             nextState_char) )

        currentState_index = nextState_index
        currentState_char = nextState_char

    if trace:
        print("Finish typing the word ", word)
        print( "char outputs:\t\t", charOutputsObservedSofar+"_" )
        print( "state trajectory:\t", stateTrajectorySofar+"F" )

    return charOutputsObservedSofar[1:]

In [ ]:
pr_hit = 0.9
pr_miss = 0.1
deg_kb = 4

pr_repeat = 0.1
pr_moveOn = 0.9
deg_sp = 4

word = "his"

#See the trace of the simulation of typing one word
typeOneWord( word, True)

First state reached after leaving I: (index, char)= (0, 'h')
Current state: (index, char)= (0, 'h')
h  is pressed when trying to type  h
char outputs so far:	 _h
state trajectory so far: Ih



NameError: ignored

In [ ]:
# See the result of typing the same word 10000 times
[typeOneWord(word, False) for i in range(10000) ]

NameError: ignored

# 2. Basics of file input/output in Python

### 2.1 Basics of file output in Python

In [ ]:
# write data into a file.
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile1","w")
for L in Lines:
    file.write(L)
file.close()

In [ ]:
# %load outputFile1

In [ ]:
# write data into a file.
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile2","w")
file.writelines(Lines)
file.close()


In [ ]:
# %load outputFile2

In [ ]:
# write data into a file.
Lines = ["Line 1: a;","Line 2: b;","Line 3: c;"]

file = open("outputFile3","w")
for line in Lines:
    file.writelines(line +"\n")
file.close()


In [ ]:
# %load outputFile3



### 2.2 Basics of file intput in Python

In [ ]:
# read data from a file.
file = open("outputFile3","r")
x=file.read()
print("x is", x)
file.close()


x is Line 1: a;
Line 2: b;
Line 3: c;



In [ ]:
file = open("outputFile3","r")
x=file.readline()
y=file.readline()
z=file.readline()
#show what we got in x, y, z
x, y, z


('Line 1: a;\n', 'Line 2: b;\n', 'Line 3: c;\n')

In [ ]:
file = open("outputFile3","r")
lines=file.readlines()
print("lines is", lines)


lines is ['Line 1: a;\n', 'Line 2: b;\n', 'Line 3: c;\n']


In [ ]:
# Strip the white spaces(i.e. spaces, tabs, and specifically'\n' in this case)
strippedLines = [line.strip( ) for line in lines]
strippedLines


['Line 1: a;', 'Line 2: b;', 'Line 3: c;']

In [ ]:
# Read words from biolaVision.txt
file = open("biolaVision.txt","r")
lines = file.readlines()
file.close()

words = [line.strip( ) for line in lines]

print(len(words), " words in biolaVision.txt:\n")
words

72  words in biolaVision.txt:



['biola',
 'university',
 'vision',
 'is',
 'to',
 'be',
 'an',
 'exemplary',
 'christian',
 'university',
 'characterized',
 'as',
 'a',
 'community',
 'of',
 'grace',
 'that',
 'promotes',
 'and',
 'inspires',
 'personal',
 'life',
 'transformation',
 'in',
 'christ',
 'which',
 'illuminates',
 'the',
 'world',
 'with',
 'his',
 'light',
 'and',
 'truth',
 'further',
 'as',
 'a',
 'global',
 'center',
 'for',
 'christian',
 'thought',
 'and',
 'an',
 'influential',
 'evangelical',
 'voice',
 'that',
 'addresses',
 'crucial',
 'cultural',
 'issues',
 'biola',
 'university',
 'aspires',
 'to',
 'lead',
 'with',
 'confidence',
 'and',
 'compassion',
 'an',
 'intellectual',
 'and',
 'spiritual',
 'renewal',
 'that',
 'advances',
 'the',
 'purpose',
 'of',
 'christ']

# 3. Implement typeOneArticle

In [ ]:
# C++: void typeOneArticle (const char * corruptedMessageFile, const char * sourceArticle,
#                           bool trace = false);
# Or
# C++: void typeOneArticle (const string corruptedMessageFile, const string sourceArticle,
#                           bool trace = false);
# ==>
# Python: typeOneArticle (corruptedMessageFile, sourceArticle,
#                           trace = False)


In [ ]:
def typeOneArticle(corruptedMessageFile, sourceArticle, trace = False):
    file = open(sourceArticle,"r")
    lines = file.readlines()
    file.close()
    words = [line.strip( ) for line in lines]

    corruptedWords = [ typeOneWord(word) for word in words]
    if trace:
        print( corruptedWords )

    file = open(corruptedMessageFile,"w")
    for corruptedWord in corruptedWords:
        file.writelines(corruptedWord+"\n")
    file.close()




In [ ]:
pr_hit = 0.6
pr_miss = 0.4
deg_kb = 2

pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2

typeOneArticle("corruptedVision", "biolaVision.txt")


In [ ]:
# %load corruptedVision

# 4. Specifics about Program 3A

In [ ]:
wordToType = "his"
observedString = "he"

# 4.1 Data Structures: Lists

## State representations (excluding the special state I and F)

In [ ]:
state_indicices = list( range(len(wordToType) ))
state_indicices

[0, 1, 2]

In [ ]:
state_chars = wordToType
state_chars

'his'

In [ ]:
states = [ (state_index, state_char) for state_index, state_char in zip(  state_indicices, state_chars)]
states

[(0, 'h'), (1, 'i'), (2, 's')]

## HMM representation (excluding the special state I)

In [ ]:
#At the beginning, the initial probability vector Pi after leaving the special state I

vector_pi_list = getPrTableForPossibleInitialStatesGivenTheWord(wordToType)
vector_pi_list

[0.5714285714285714, 0.2857142857142857, 0.14285714285714285]

In [ ]:
# Transition probability matrix A (excluding the rows for I and F and the column for I)
lenthOfWord = len(wordToType)
matrix_A_List = [ getPrTableForPossibleNextStatesGivenWord(wordToType, currentState) for currentState in range(lenthOfWord)]
matrix_A_List

[[0.2, 0.4571428571428572, 0.2285714285714286, 0.1142857142857143],
 [0.0, 0.2, 0.5333333333333333, 0.26666666666666666],
 [0.0, 0.0, 0.2, 0.8]]

In [ ]:
# Access the transition probability $a_{ij}$
i = 0; j =1
matrix_A_List[i][j]

0.4571428571428572

In [ ]:
# observation probability matrix B,
# excluding the rows for I and F columns,
# exclucing the columns for ReadyToType and EndOfWord
alphabet = "abcdefghijklmnopqrstuvwxyz"
matrix_B_List = [ [prCharGiveCharState(char, state_Char) for char in alphabet]
                  for state_Char in state_chars]
matrix_B_List

[[0.001562786154691411,
  0.003125572309382822,
  0.006251144618765644,
  0.012502289237531288,
  0.025004578475062576,
  0.05000915695012515,
  0.1000183139002503,
  0.6,
  0.1000183139002503,
  0.05000915695012515,
  0.025004578475062576,
  0.012502289237531288,
  0.006251144618765644,
  0.003125572309382822,
  0.001562786154691411,
  0.0007813930773457055,
  0.00039069653867285274,
  0.00019534826933642637,
  9.767413466821319e-05,
  4.883706733410659e-05,
  2.4418533667053296e-05,
  4.883706733410659e-05,
  9.767413466821319e-05,
  0.00019534826933642637,
  0.00039069653867285274,
  0.0007813930773457055],
 [0.0007813930773457055,
  0.001562786154691411,
  0.003125572309382822,
  0.006251144618765644,
  0.012502289237531288,
  0.025004578475062576,
  0.05000915695012515,
  0.1000183139002503,
  0.6,
  0.1000183139002503,
  0.05000915695012515,
  0.025004578475062576,
  0.012502289237531288,
  0.006251144618765644,
  0.003125572309382822,
  0.001562786154691411,
  0.0007813930773457

In [ ]:
# Access the observation probability $b_{ij}$
i = 0; j =0   #Typing the first character (state_index is 0) in the word but get 'a' generated
matrix_B_List[i][j]

0.001562786154691411

In [ ]:
# Access the observation probability $b_{ij}$
i = 0; j =1   #Typing the first character (state_index is 0) in the word but get 'b' generated
matrix_B_List[i][j]

0.003125572309382822

# 4.2 Data Structures: Use np arrays as data structures instead

In [ ]:
vector_pi = np.array( vector_pi_list )
matrix_A = np.array( matrix_A_List )
matrix_B = np.array( matrix_B_List )

In [ ]:
print(vector_pi)  #Check the shape
print(vector_pi.shape)  #Check the shape
vector_pi.sum()         #Check the sum of probabilties

[0.57142857 0.28571429 0.14285714]
(3,)


1.0

In [ ]:
print(matrix_A.shape)  #Check the shape
matrix_A.sum(axis = 1) #Check the sum of probabilties on each row

(3, 4)


array([1., 1., 1.])

In [ ]:
print(matrix_B.shape)  #Check the shape
matrix_B.sum(axis = 1) #Check the sum of probabilties on each row

(3, 26)


array([1., 1., 1.])

# 4.3 Ideas about implemening the forward algorithm for HMMs using Numpy arrays

## Example: Use the forward alforithm to figure out the probability of seeing "ab" (i.e "ReadToType_ab_EndOfWord"when the person is try to type the word "his".

## You can use lists above as the data structures to implement the work or You may consider using numpy arrays as the data structures to implement the work. In the following, we show the basic ideas of using Numpy arrays for the implementation of the forward algorithm to figure out the probability.

## Note that in general the number of stages invloved should be 2+the number of characters in the observed string. In this case, we have 4 stages only since there are only 2 chracters observed.

In [ ]:
# Paraemter values for the 1D keyboad are stored in global variables; you can change them
pr_hit = 0.6
pr_miss = 0.4
deg_kb = 2

pr_repeat = 0.2
pr_moveOn = 0.8
deg_sp = 2

wordToType = "his"
observedString = "ab"


## Stage 1 (The column for observing ReadyToType ): Must start at state I (probability 0 for the other states)

## Stage 2 (The column for observing 'a'): only keep the real states (excluding I and F)

In [ ]:
#Probabilties of transitioning into each of the real states from I
print( vector_pi.shape)
vector_pi

(3,)


array([0.57142857, 0.28571429, 0.14285714])

In [ ]:
#Probabilties of transitioning into each of the real states from I
stage2_1 = vector_pi

In [ ]:
#Probabilities of observing the first character in each of the real states (excluding F)
observationIndex = 0
charObserved = observedString[observationIndex]
print( "charObserved:", charObserved)
indexOfObservedChar = ord(charObserved) - ord('a')
matrix_B[:, indexOfObservedChar]

charObserved: a


array([0.00156279, 0.00078139, 0.00078139])

In [ ]:
# Combine the two items above to calculate
# Probabilities of ending in each of the real states (excluding F) and observing a
stage2_2 = stage2_1*matrix_B[:, indexOfObservedChar]
stage2_2

array([0.00089302, 0.00022326, 0.00011163])

## Stage 3 (The column for observing 'b'): only keep the real states (excluding I and F)

In [ ]:
stage2_2

array([0.00089302, 0.00022326, 0.00011163])

In [ ]:
stage2_2[:, np.newaxis]
print( stage2_2[:, np.newaxis].shape )
stage2_2[:, np.newaxis]

(3, 1)


array([[0.00089302],
       [0.00022326],
       [0.00011163]])

In [ ]:
#Probabilties of further transitioning into each of the real states
stage3_1 = stage2_2[:, np.newaxis]  * matrix_A
stage3_1

array([[1.78604132e-04, 4.08238016e-04, 2.04119008e-04, 1.02059504e-04],
       [0.00000000e+00, 4.46510330e-05, 1.19069421e-04, 5.95347107e-05],
       [0.00000000e+00, 0.00000000e+00, 2.23255165e-05, 8.93020660e-05]])

In [ ]:
stage3_2 = stage3_1.sum(axis = 0)
stage3_2

array([0.0001786 , 0.00045289, 0.00034551, 0.0002509 ])

In [ ]:
#Probabilities of ending in each of the real states (excluding F)
stage3_2[:-1]

array([0.0001786 , 0.00045289, 0.00034551])

In [ ]:
#Probabilities of observing the second character in each of the real states (excluding F)
observationIndex = 1
charObserved = observedString[observationIndex]
print( "charObserved:", charObserved)
indexOfObservedChar = ord(charObserved) - ord('a')
matrix_B[:, indexOfObservedChar]


charObserved: b


array([0.00312557, 0.00156279, 0.0003907 ])

In [ ]:
# Combine the two items above to calculate
# Probabilities of ending in each of the real states (excluding F) and observing b
stage3_3 = stage3_2[:-1]*matrix_B[:, indexOfObservedChar]
stage3_3

array([5.58240129e-07, 7.07768735e-07, 1.34991103e-07])

## Stage 4 (The column for observing EndOfWord ): Must be in the Final State F (probability = 0 for the other states)

In [ ]:
# Probabilities of ending in each of the real states (excluding F) and observing b
stage3_3

array([5.58240129e-07, 7.07768735e-07, 1.34991103e-07])

In [ ]:
# Probabilities of then transitioning to F from each of the real states
matrix_A[:, len( wordToType)]

array([0.11428571, 0.26666667, 0.8       ])

In [ ]:
# Combine the two items above to calculate the
# Probabilities of then transition to F and ending the typing process of the word
# Transisiton to F (index 3 in this case )
stage4_1 = stage3_3 * matrix_A[:, len( wordToType)]
stage4_1

array([6.37988719e-08, 1.88738329e-07, 1.07992882e-07])

In [ ]:
# The result is the sum of all these probabilities

In [ ]:
result = stage4_1.sum()
result

3.6053008344833416e-07

## The result above is also what you would get from the sample demo program for HMMs.
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *

# 4.4 Complete the implementation of the forward algorithm below ==>
## Define a function prOf1CharSeriesWhenTyping1Word_F to generalize the process (of the forward algorithm) demonstrated in the case above for any given wordToType and any given observedString

In [ ]:
# Using the forward algorithm algorithm to determine the probability:
# The function should calculate and return
#     the probability of getting the string d
#     when the user (modelled by the parameter values of pr_hit, pr_repeat, degenerate_kb, ...)
#     want to type the word in string w
# When the trace is True, the function will report the trace of computation done.

def prOf1CharSeriesWhenTyping1Word_F(observedString, wordToType, trace = False):
    #The probability distribution after leaving I
    vector_pi_list = getPrTableForPossibleInitialStatesGivenTheWord(wordToType)

    #The transition probability matrix A
    lenthOfWord = len(wordToType)
    matrix_A_List = [ getPrTableForPossibleNextStatesGivenWord(wordToType, currentState)
                      for currentState in range(lenthOfWord)]

    #The observation probability matrix B
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    matrix_B_List = [ [prCharGiveCharState(char, state_Char) for char in alphabet]
                      for state_Char in wordToType]

    ##############################################
    #Cast them into numpy arrays
    ##############################################
    vector_pi = np.array( vector_pi_list )
    matrix_A = np.array( matrix_A_List )
    matrix_B = np.array( matrix_B_List )

    if trace == True:
        print("vector_pi", vector_pi.shape, ":\n", vector_pi)
        print("matrix_A", matrix_A.shape, ":\n", matrix_A)
        print("matrix_B", matrix_B.shape, ":\n", matrix_B)

    ##############################################
    #For the first column (corresponding to the first character observed)
    ##############################################
    observationIndex = 0
    charObserved = observedString[observationIndex]
    if trace == True:
        print( "\n\nobservationIndex, charObserved:", observationIndex, ",", charObserved)
    indexOfObservedCharInAlphabet = ord(charObserved) - ord('a');

    #transitionProbabilties record the 1st-stage results of a column regarding
    #    the probabilities of ending in each of the states at this point
    transitionProbabilties = vector_pi
    if trace == True:
        print("Probabilties of ending at the states at this point:\n", transitionProbabilties)

    #columnProbabilities record the 2nd-stage results of a column regarding
    #    the probabilities of ending in each of the states at this point and also
    #                         seeing the specific character at this point
    observationProbabilities = matrix_B[:, indexOfObservedCharInAlphabet]
    if trace == True:
        print("probabilities of observing ", charObserved, " at specific states alone:\n",
              observationProbabilities)

    columnProbabilities = transitionProbabilties * observationProbabilities
    if trace == True:
        print("Probabilties of observing up to", charObserved,
              " and ending at the states at this point:\n", columnProbabilities)


    ##############################################
    # For the remaining columns one at a time
    ##############################################
    lenthOfObservedString = len(observedString)
    for observationIndex in np.arange(1, lenthOfObservedString):
        charObserved = observedString[observationIndex]
        if trace == True:
            print( "\nobservationIndex, charObserved:", observationIndex, ",", charObserved)

        ##############################################################
        ## write the reamining part of the body of the for loop
        ##############################################################
        # observationProbabilities = matrix_B[:, indexOfObservedCharInAlphabet]
        if trace == True:
            print("probabilities of observing ", charObserved, " at specific states alone:\n",
              observationProbabilities)
        transitionProbabilities2 = columnProbabilities[:, np.newaxis] * matrix_A
        transitionProbabilities = transitionProbabilities2.sum(axis = 0)
        #if trace == True:
           # print("Transition probabilities at this point: ", transitionProbabilities2)
        columnProbabilities = observationProbabilities * transitionProbabilities[:-1]

        if trace == True:
            print("Probabilities of observing up to", charObserved, " and ending at the states at this point:\n ", columnProbabilities)

        # for observationIndex in np.arrange(1, lenthOfObservedString)
        # columnProbabilities[:, np.newaxis]
        # columnProbabilities2 = columnPorbabilities[:, np.newaxis] * matrix_A.sum
        # columnProbabilities = columnProbabilities2[:-1] * matrix_B[:, indexOfObservedChar]
        # ...
        # ...

    ##############################################
    # Determine the sum of probabilities of transitioning to the Final state F from each state
    ##############################################
    if trace == True:
        print("\nprobabilities of transitioning to F from states at this point:\n",
              matrix_A[:, len( wordToType)] )
    pr = (columnProbabilities * matrix_A[:, len( wordToType)]).sum()
    if trace == True:
        print("\nSum of the probabilitie above:", pr);

    return pr


In [ ]:
prOf1CharSeriesWhenTyping1Word_F("bbd", "abc", True)

vector_pi (3,) :
 [0.57142857 0.28571429 0.14285714]
matrix_A (3, 4) :
 [[0.2        0.45714286 0.22857143 0.11428571]
 [0.         0.2        0.53333333 0.26666667]
 [0.         0.         0.2        0.8       ]]
matrix_B (3, 26) :
 [[6.00000000e-01 1.00018314e-01 5.00091570e-02 2.50045785e-02
  1.25022892e-02 6.25114462e-03 3.12557231e-03 1.56278615e-03
  7.81393077e-04 3.90696539e-04 1.95348269e-04 9.76741347e-05
  4.88370673e-05 2.44185337e-05 4.88370673e-05 9.76741347e-05
  1.95348269e-04 3.90696539e-04 7.81393077e-04 1.56278615e-03
  3.12557231e-03 6.25114462e-03 1.25022892e-02 2.50045785e-02
  5.00091570e-02 1.00018314e-01]
 [1.00018314e-01 6.00000000e-01 1.00018314e-01 5.00091570e-02
  2.50045785e-02 1.25022892e-02 6.25114462e-03 3.12557231e-03
  1.56278615e-03 7.81393077e-04 3.90696539e-04 1.95348269e-04
  9.76741347e-05 4.88370673e-05 2.44185337e-05 4.88370673e-05
  9.76741347e-05 1.95348269e-04 3.90696539e-04 7.81393077e-04
  1.56278615e-03 3.12557231e-03 6.25114462e-03 1.25

0.00298575154160429

In [ ]:
# With trace
prOf1CharSeriesWhenTyping1Word_F("his", "his", True)

vector_pi (3,) :
 [0.57142857 0.28571429 0.14285714]
matrix_A (3, 4) :
 [[0.2        0.45714286 0.22857143 0.11428571]
 [0.         0.2        0.53333333 0.26666667]
 [0.         0.         0.2        0.8       ]]
matrix_B (3, 26) :
 [[1.56278615e-03 3.12557231e-03 6.25114462e-03 1.25022892e-02
  2.50045785e-02 5.00091570e-02 1.00018314e-01 6.00000000e-01
  1.00018314e-01 5.00091570e-02 2.50045785e-02 1.25022892e-02
  6.25114462e-03 3.12557231e-03 1.56278615e-03 7.81393077e-04
  3.90696539e-04 1.95348269e-04 9.76741347e-05 4.88370673e-05
  2.44185337e-05 4.88370673e-05 9.76741347e-05 1.95348269e-04
  3.90696539e-04 7.81393077e-04]
 [7.81393077e-04 1.56278615e-03 3.12557231e-03 6.25114462e-03
  1.25022892e-02 2.50045785e-02 5.00091570e-02 1.00018314e-01
  6.00000000e-01 1.00018314e-01 5.00091570e-02 2.50045785e-02
  1.25022892e-02 6.25114462e-03 3.12557231e-03 1.56278615e-03
  7.81393077e-04 3.90696539e-04 1.95348269e-04 9.76741347e-05
  4.88370673e-05 2.44185337e-05 4.88370673e-05 9.76

0.0011539716174544866

In [ ]:
# No trace
prOf1CharSeriesWhenTyping1Word_F("his", "his")

0.0011539716174544866

In [ ]:
prOf1CharSeriesWhenTyping1Word_F("sasdfsdfsdfsdfsdfsfafsdfdasff", "sdfsdfsdfsfparameters")

1.508716565989372e-25